In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

from nltk import ngrams
from wordcloud import WordCloud
from collections import Counter

from src.constants import *

In [2]:
with open(os.path.join(DATA_PATH, "2021AB_SN", "SG"), "r") as f:
    group_df = pd.read_csv(f, sep="|", header=None)

full_names = {}
for abbrev, name in group_df[[0,1]].itertuples(index=False):
    if abbrev not in full_names.keys():
        full_names[abbrev] = name.upper()

In [3]:
total_distrib = {}
tokens = []

for filename in os.listdir(ARTICLES_PATH):
    
    with open(os.path.join(ARTICLES_PATH, filename, "entities.csv"), "r") as f:
        entities_df = pd.read_csv(f).drop("Unnamed: 0", axis=1)
    
    for i, v in entities_df["Type"].value_counts().iteritems():
        if i == "ENTITY":
            i = "UNDEF"
        elif i in full_names.keys():
            i = full_names[i]
        if i not in total_distrib.keys():
            total_distrib[i] = v
        else:
            total_distrib[i] += v
    
    tokens += list(entities_df["Word"].astype(str).values)

list_distrib = []
for name, v in total_distrib.items():
    list_distrib.append({"group": name, "frequency": v})

In [4]:
counted = Counter(tokens)
counted_2 = Counter(ngrams(tokens, 2))
counted_3 = Counter(ngrams(tokens, 3))

word_freq = pd.DataFrame(counted.items(), columns=['word', 'frequency']).sort_values(by='frequency', ascending=False)
word_pairs = pd.DataFrame(counted_2.items(),columns=['pairs', 'frequency']).sort_values(by='frequency', ascending=False)
trigrams = pd.DataFrame(counted_3.items(), columns=['trigrams', 'frequency']).sort_values(by='frequency', ascending=False)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokens)
vectorizer.get_feature_names_out()

array(['00', '000', '00001', ..., 'πrd', 'χ2', 'ᶧproblem'], dtype=object)

In [6]:
X.shape

(151162, 10793)